## 네이버 금융 개별종목 수집

1. 수집 하고자 하는 페이지의 URL을 알아본다.
1. 파이썬의 작은 브라우저 requests 를 통해 URL에 접근한다.
1. response.status_code 가 200 OK 라면 정상 응답입니다.
1. request의 response 값에서 response.text 만 받아옵니다.
1. html 텍스트를 받아왔다면  BeautifulSoup 로 html 을 해석합니다.
1. soup.select 를 통해 원하는 태그에 접근합니다.
1. 목록을 먼저 받아옵니다.
1. 목록에서 행을 하나씩 가져옵니다.
1. 행을 모아서 데이터프레임으로 만듭니다.
=> 판다스라면 read_html로 table로 된 태그에 대해 위 과정을 코드 한 줄로 해줍니다.



### Keyword

* html 파일 읽어오기
    * pd.read_html(url, encoding="cp949")

* 결측 데이터 제거하기(axis 0:행, 1:열)
    * table[0].dropna()

* 데이터 프레임 합치기
    * pd.concat([df1, df2, df3])

* 중복데이터 제거
    * df.drop_duplicates()

* 과학적 기수법
    * 1.210000e+02 => 121

* 날짜 column의 첫 row값 확인
    * date = df.iloc[0]["날짜"]

* 파일로 저장하기 
    * df.to_csv(file_name, index=False)

* 파일 읽어오기
    * pd.read_csv(file_name)

## 수집할 페이지 보기

* 네이버 금융 국내증시 : https://finance.naver.com/sise/
* 주요 종목
    * 삼성전자 : https://finance.naver.com/item/main.naver?code=005930
    * 현대차 : https://finance.naver.com/item/main.naver?code=005380
    * SK하이닉스 : https://finance.naver.com/item/main.naver?code=000660

## 라이브러리 로드

In [1]:
# 라이브러리 로드
import pandas as pd
import numpy as np

## 수집할 URL 정하기

In [2]:
# 종목번호와 상장사 이름을 item_code와 item_name으로 설정
# item_code = "323410"
# item_name = "카카오뱅크"

item_code = "005930"
item_name = "삼성전자"

# 종목 URL 만들기
url = f'https://finance.naver.com/item/sise_day.naver?code={item_code}&page=1'
print(url)

https://finance.naver.com/item/sise_day.naver?code=005930&page=1


## requests를 통한 HTTP 요청
* [Requests: HTTP for Humans™ — Requests documentation](https://requests.readthedocs.io/en/master/)
* [Quickstart — Requests documentation # custom-headers](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

In [3]:
import requests
# 헤더스 필요!!!
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}
response = requests.get(url, headers = headers)
response.status_code

200

In [4]:
response.text

'\n<html lang="ko">\n<head>\n<meta http-equiv="Content-Type" content="text/html; charset=euc-kr">\n<title>네이버 금융</title>\n\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/common.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/layout.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/main.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock2.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock3.css">\n<link rel="stylesheet" type="text/css" href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/world.css">\n</head>\n<body>\n<script

## BeautifulSoup 을 통한 table 태그 찾기

In [5]:
from bs4 import BeautifulSoup as bs

html = bs(response.text, "lxml")
html

<html lang="ko">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<title>네이버 금융</title>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/common.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/layout.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/main.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock2.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/newstock3.css" rel="stylesheet" type="text/css"/>
<link href="https://ssl.pstatic.net/imgstock/static.pc/20220511192542/css/world.css" rel="stylesheet" type="text/css"/>
</head>
<body>
<script language="

In [6]:
html.title.name

'title'

In [7]:
html.title.string

'네이버 금융'

In [8]:
html.title.parent.name

'head'

In [9]:
print(len(html.find_all("table")))
html.find_all("table")

2


[<table cellspacing="0" class="type2">
 <tr>
 <th>날짜</th>
 <th>종가</th>
 <th>전일비</th>
 <th>시가</th>
 <th>고가</th>
 <th>저가</th>
 <th>거래량</th>
 </tr>
 <tr>
 <td colspan="7" height="8"></td>
 </tr>
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2022.05.18</span></td>
 <td class="num"><span class="tah p11">68,600</span></td>
 <td class="num">
 <img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
 				1,000
 				</span>
 </td>
 <td class="num"><span class="tah p11">68,300</span></td>
 <td class="num"><span class="tah p11">68,600</span></td>
 <td class="num"><span class="tah p11">68,100</span></td>
 <td class="num"><span class="tah p11">2,255,246</span></td>
 </tr>
 <tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
 <td align="center"><span class="tah p10 gray03">2022.05.17</span></td>
 <td class="num"><span class="

In [10]:
print(len(html.find("table")))
html.find("table")

33


<table cellspacing="0" class="type2">
<tr>
<th>날짜</th>
<th>종가</th>
<th>전일비</th>
<th>시가</th>
<th>고가</th>
<th>저가</th>
<th>거래량</th>
</tr>
<tr>
<td colspan="7" height="8"></td>
</tr>
<tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2022.05.18</span></td>
<td class="num"><span class="tah p11">68,600</span></td>
<td class="num">
<img alt="상승" height="6" src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" style="margin-right:4px;" width="7"/><span class="tah p11 red02">
				1,000
				</span>
</td>
<td class="num"><span class="tah p11">68,300</span></td>
<td class="num"><span class="tah p11">68,600</span></td>
<td class="num"><span class="tah p11">68,100</span></td>
<td class="num"><span class="tah p11">2,255,246</span></td>
</tr>
<tr onmouseout="mouseOut(this)" onmouseover="mouseOver(this)">
<td align="center"><span class="tah p10 gray03">2022.05.17</span></td>
<td class="num"><span class="tah p11">67,600</span></td>
<

In [11]:
tables = html.select("table")
tables == html.findAll("table")

True

In [12]:
len(tables)

2

In [13]:
html.table == html.find("table")

True

## pandas 코드 한 줄로 데이터 수집하기

In [14]:
# table = pd.read_html(requests.get(url, headers = {"User-agent" : "Mozilla/5.0"}, encoding = "cp494"))

In [15]:
# read_html을 이용하여 url의 page내의 값을 DataFrame으로 받아옵니다.
# cp949는 한글 인코딩을 위해 사용합니다. 기본 인코딩 설정은 utf-8 이며, 
# 네이버의 일별 시세는 cp949 인코딩으로 불러올 수 있습니다.
# 데이터를 로드 했을 때 한글 인코딩이 깨진다면 대부분 cp949 로 불러올 수 있습니다.
table = pd.read_html(str(tables), encoding = "cp949")
print(len(table))
table[0]


2


,날짜,종가,전일비,시가,고가,저가,거래량
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2022.05.18,68600.0,1000.0,68300.0,68600.0,68100.0,2255246.0
2,2022.05.17,67600.0,1300.0,66600.0,67900.0,66600.0,15680447.0
3,2022.05.16,66300.0,200.0,67100.0,67400.0,66100.0,11937555.0
4,2022.05.13,66500.0,1600.0,65300.0,66700.0,65200.0,14551536.0
5,2022.05.12,64900.0,800.0,65200.0,65500.0,64900.0,16414188.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2022.05.11,65700.0,0.0,65500.0,66300.0,65200.0,12330920.0


In [16]:
# table[0]와 table[1]을 확인하여 보면 table[0]에 필요한 데이터들이 있습니다.
table[1]

,0,1,2,3,4,5,6,7,8,9,10,11
0,1,2,3,4,5,6,7,8,9,10,다음,맨뒤


In [17]:
# dropna를 통해 결측치가 들어있는 row를 제거합니다.
temp = table[0].dropna()
temp

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.05.18,68600.0,1000.0,68300.0,68600.0,68100.0,2255246.0
2,2022.05.17,67600.0,1300.0,66600.0,67900.0,66600.0,15680447.0
3,2022.05.16,66300.0,200.0,67100.0,67400.0,66100.0,11937555.0
4,2022.05.13,66500.0,1600.0,65300.0,66700.0,65200.0,14551536.0
5,2022.05.12,64900.0,800.0,65200.0,65500.0,64900.0,16414188.0
9,2022.05.11,65700.0,0.0,65500.0,66300.0,65200.0,12330920.0
10,2022.05.10,65700.0,400.0,65900.0,66300.0,65300.0,17235605.0
11,2022.05.09,66100.0,400.0,66300.0,66900.0,66100.0,11858736.0
12,2022.05.06,66500.0,1400.0,67000.0,67100.0,66500.0,14356156.0
13,2022.05.04,67900.0,400.0,68000.0,68400.0,67500.0,11505248.0


## 페이지별 데이터 수집 함수 만들기

In [18]:
print(url)

https://finance.naver.com/item/sise_day.naver?code=005930&page=1


In [19]:
# 종목 번호를 이용해 page에 따라 데이터를 읽어오는 함수
# """ 는 이 두개 사이의 행들은 주석 처리되며, 함수의 docstring 으로 사용됩니다.

def get_day_list(item_code, page_no):
    """
    일자별 시세를 페이지별로 수집
    """ 
    url = f"https://finance.naver.com/item/sise_day.naver?code={item_code}&page={page_no}"
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.67 Safari/537.36'}
    response = requests.get(url, headers = headers)
    html = bs(response.text, "lxml")
    tables = html.select("table")
    table = pd.read_html(str(tables), encoding = "cp949") # 굳이 인코딩
    temp = table[0].dropna()
    return temp
    
    

In [20]:
# 함수가 잘 만들어졌는지 확인하기
page_no = 1
get_day_list(item_code, page_no)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.05.18,68600.0,1000.0,68300.0,68600.0,68100.0,2255246.0
2,2022.05.17,67600.0,1300.0,66600.0,67900.0,66600.0,15680447.0
3,2022.05.16,66300.0,200.0,67100.0,67400.0,66100.0,11937555.0
4,2022.05.13,66500.0,1600.0,65300.0,66700.0,65200.0,14551536.0
5,2022.05.12,64900.0,800.0,65200.0,65500.0,64900.0,16414188.0
9,2022.05.11,65700.0,0.0,65500.0,66300.0,65200.0,12330920.0
10,2022.05.10,65700.0,400.0,65900.0,66300.0,65300.0,17235605.0
11,2022.05.09,66100.0,400.0,66300.0,66900.0,66100.0,11858736.0
12,2022.05.06,66500.0,1400.0,67000.0,67100.0,66500.0,14356156.0
13,2022.05.04,67900.0,400.0,68000.0,68400.0,67500.0,11505248.0


In [21]:
len(table)

2

In [44]:
html.select("a")[-1]["href"].split('=')

['/item/sise_day.naver?code', '005930&page', '651']

## 반복문을 통한 전체 일자 데이터 수집하기
* (주의) 기간이 긴 데이터를 수집할때는 서버에 부담을 주지 않기 위해 time.sleep()값을 주세요.

In [22]:
import time

# 데이터를 저장할 빈 변수 선언
item_list = []
for page_no in range(1, 5):
    item_list.append(get_day_list(item_code, page_no))
    time.sleep(0.1)
    
item_list


[            날짜       종가     전일비       시가       고가       저가         거래량
 1   2022.05.18  68600.0  1000.0  68300.0  68600.0  68100.0   2255246.0
 2   2022.05.17  67600.0  1300.0  66600.0  67900.0  66600.0  15680447.0
 3   2022.05.16  66300.0   200.0  67100.0  67400.0  66100.0  11937555.0
 4   2022.05.13  66500.0  1600.0  65300.0  66700.0  65200.0  14551536.0
 5   2022.05.12  64900.0   800.0  65200.0  65500.0  64900.0  16414188.0
 9   2022.05.11  65700.0     0.0  65500.0  66300.0  65200.0  12330920.0
 10  2022.05.10  65700.0   400.0  65900.0  66300.0  65300.0  17235605.0
 11  2022.05.09  66100.0   400.0  66300.0  66900.0  66100.0  11858736.0
 12  2022.05.06  66500.0  1400.0  67000.0  67100.0  66500.0  14356156.0
 13  2022.05.04  67900.0   400.0  68000.0  68400.0  67500.0  11505248.0,
             날짜       종가     전일비       시가       고가       저가         거래량
 1   2022.05.03  67500.0   200.0  67400.0  68400.0  67300.0  14168875.0
 2   2022.05.02  67300.0   100.0  66600.0  67600.0  66500.0  14

In [123]:
# 카카오페이 377300
# 왜 안될까?????? # 해결!!!!!
# 복습할때 꼭 복습 해보기
# naver financial
item_code = "377300"
page_no = 1
item_list = []

while True:    
    if get_day_list(item_code, page_no).to_dict() == get_day_list(item_code, page_no + 1).to_dict():
        item_list.append(get_day_list(item_code, page_no))
        break
    else:
        item_list.append(get_day_list(item_code, page_no))
        time.sleep(0.01)
        page_no += 1
    
item_list


[            날짜        종가     전일비        시가        고가        저가       거래량
 1   2022.05.18   90100.0  1400.0   89600.0   90900.0   89300.0  108398.0
 2   2022.05.17   88700.0  2900.0   85800.0   88700.0   85400.0  214643.0
 3   2022.05.16   85800.0   200.0   86500.0   88700.0   85700.0  277584.0
 4   2022.05.13   86000.0   100.0   85900.0   86600.0   85000.0  307400.0
 5   2022.05.12   85900.0  5500.0   90000.0   90000.0   85000.0  512917.0
 9   2022.05.11   91400.0   500.0   91000.0   92800.0   89700.0  333197.0
 10  2022.05.10   91900.0  1900.0   91200.0   92000.0   89700.0  514324.0
 11  2022.05.09   93800.0  4000.0   96000.0   96700.0   93400.0  462857.0
 12  2022.05.06   97800.0  8700.0  104500.0  105000.0   97100.0  735556.0
 13  2022.05.04  106500.0  2000.0  108000.0  109500.0  106000.0  302514.0,
             날짜        종가     전일비        시가        고가        저가       거래량
 1   2022.05.03  108500.0   500.0  108000.0  113500.0  106000.0  738570.0
 2   2022.05.02  108000.0  4500.0  10

In [24]:
page_no = 1
item_code = "377300"
item_name = "카카오페이"
get_day_list(item_code, page_no)

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.05.18,89900.0,1200.0,89600.0,90400.0,89300.0,39270.0
2,2022.05.17,88700.0,2900.0,85800.0,88700.0,85400.0,214643.0
3,2022.05.16,85800.0,200.0,86500.0,88700.0,85700.0,277584.0
4,2022.05.13,86000.0,100.0,85900.0,86600.0,85000.0,307400.0
5,2022.05.12,85900.0,5500.0,90000.0,90000.0,85000.0,512917.0
9,2022.05.11,91400.0,500.0,91000.0,92800.0,89700.0,333197.0
10,2022.05.10,91900.0,1900.0,91200.0,92000.0,89700.0,514324.0
11,2022.05.09,93800.0,4000.0,96000.0,96700.0,93400.0,462857.0
12,2022.05.06,97800.0,8700.0,104500.0,105000.0,97100.0,735556.0
13,2022.05.04,106500.0,2000.0,108000.0,109500.0,106000.0,302514.0


In [87]:
import time
# web page 시작번호
page_no = 1
# 데이터를 저장할 빈 변수 선언
item_list = []
# item_code = "373220"
# item_name = "LG에너지솔루션"
item_code = "377300"
item_name = "카카오페이"

prev = ""

while True:
    print(page_no)
    
    df_one_page = get_day_list(item_code, page_no)
    # 마지막 날짜를 가져옴
    curr = df_one_page.iloc[-1, 0]
    # 마지막 날짜를 비교했을 때 같으면 반복문을 종료
    if curr == prev:
        break
        
    item_list.append(df_one_page)
    page_no = page_no + 1
    # 현재 날짜를 이전 날짜 변수에 담아서 다음번에 비교합니다.
    prev = curr
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


In [124]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = pd.concat(item_list)
df

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.05.18,90100.0,1400.0,89600.0,90900.0,89300.0,108398.0
2,2022.05.17,88700.0,2900.0,85800.0,88700.0,85400.0,214643.0
3,2022.05.16,85800.0,200.0,86500.0,88700.0,85700.0,277584.0
4,2022.05.13,86000.0,100.0,85900.0,86600.0,85000.0,307400.0
5,2022.05.12,85900.0,5500.0,90000.0,90000.0,85000.0,512917.0
...,...,...,...,...,...,...,...
13,2021.11.09,147000.0,6500.0,153500.0,157500.0,146000.0,892617.0
1,2021.11.08,153500.0,16500.0,168500.0,169500.0,152000.0,1394625.0
2,2021.11.05,170000.0,1000.0,167500.0,179000.0,167500.0,1597937.0
3,2021.11.04,169000.0,24000.0,190000.0,191000.0,166000.0,3487030.0


## 수집한 데이터 하나의 데이터프레임으로 합치기

<img src="https://pandas.pydata.org/docs/_images/merging_concat_basic.png">

* [Merge, join, concatenate and compare documentation](https://pandas.pydata.org/docs/user_guide/merging.html#merge-join-concatenate-and-compare)

In [125]:
# DataFrame들이 list의 형태로 저장된 것을 concat을 이용하여 합치면서 하나의 DataFrame으로 만듭니다.
df = pd.concat(item_list)
df

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.05.18,90100.0,1400.0,89600.0,90900.0,89300.0,108398.0
2,2022.05.17,88700.0,2900.0,85800.0,88700.0,85400.0,214643.0
3,2022.05.16,85800.0,200.0,86500.0,88700.0,85700.0,277584.0
4,2022.05.13,86000.0,100.0,85900.0,86600.0,85000.0,307400.0
5,2022.05.12,85900.0,5500.0,90000.0,90000.0,85000.0,512917.0
...,...,...,...,...,...,...,...
13,2021.11.09,147000.0,6500.0,153500.0,157500.0,146000.0,892617.0
1,2021.11.08,153500.0,16500.0,168500.0,169500.0,152000.0,1394625.0
2,2021.11.05,170000.0,1000.0,167500.0,179000.0,167500.0,1597937.0
3,2021.11.04,169000.0,24000.0,190000.0,191000.0,166000.0,3487030.0


In [126]:
%timeit df.iloc[0, 0] # 이게 더 빠름

37.8 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [127]:
%timeit df.iloc[0][0]

137 µs ± 5.76 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


<img src="https://pandas.pydata.org/docs/_images/02_io_readwrite.svg">

In [130]:
# head와 tail로 데이터의 일부를 가져와서 봅니다.
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량
1,2022.05.18,90100.0,1400.0,89600.0,90900.0,89300.0,108398.0
2,2022.05.17,88700.0,2900.0,85800.0,88700.0,85400.0,214643.0
3,2022.05.16,85800.0,200.0,86500.0,88700.0,85700.0,277584.0
4,2022.05.13,86000.0,100.0,85900.0,86600.0,85000.0,307400.0
5,2022.05.12,85900.0,5500.0,90000.0,90000.0,85000.0,512917.0


In [129]:
df.tail()

,날짜,종가,전일비,시가,고가,저가,거래량
13,2021.11.09,147000.0,6500.0,153500.0,157500.0,146000.0,892617.0
1,2021.11.08,153500.0,16500.0,168500.0,169500.0,152000.0,1394625.0
2,2021.11.05,170000.0,1000.0,167500.0,179000.0,167500.0,1597937.0
3,2021.11.04,169000.0,24000.0,190000.0,191000.0,166000.0,3487030.0
4,2021.11.03,193000.0,13000.0,180000.0,230000.0,173000.0,11799881.0


## 데이터프레임에 종목코드와 종목명을 추가하기
* 파생변수 만들기

In [131]:
# '종목코드'와 '종목명' column을 추가하면서 각각 item_code와 item_name 값을 입력합니다.
df["종목코드"] = item_code
df["종목명"] = item_name

In [132]:
df.head()

,날짜,종가,전일비,시가,고가,저가,거래량,종목코드,종목명
1,2022.05.18,90100.0,1400.0,89600.0,90900.0,89300.0,108398.0,377300,카카오뱅크
2,2022.05.17,88700.0,2900.0,85800.0,88700.0,85400.0,214643.0,377300,카카오뱅크
3,2022.05.16,85800.0,200.0,86500.0,88700.0,85700.0,277584.0,377300,카카오뱅크
4,2022.05.13,86000.0,100.0,85900.0,86600.0,85000.0,307400.0,377300,카카오뱅크
5,2022.05.12,85900.0,5500.0,90000.0,90000.0,85000.0,512917.0,377300,카카오뱅크


## 컬럼 순서 변경하기

In [133]:
# DataFrame에서 column 들의 이름을 순서를 조정하여 column순서를 변경할 수 있습니다.
cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
df = df[cols]
df.head(1)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
1,377300,카카오뱅크,2022.05.18,90100.0,1400.0,89600.0,90900.0,89300.0,108398.0


In [134]:
import FinanceDataReader as fdr

fdr.DataReader("FB")

,Close,Open,High,Low,Volume,Change
Date,,,,,,
2012-05-18,38.23,42.05,45.00,38.00,580590000.0,-0.5762
2012-05-21,34.03,36.53,36.66,33.00,168310000.0,-0.1099
2012-05-22,31.00,32.61,33.59,30.94,102050000.0,-0.0890
2012-05-23,32.00,31.37,32.50,31.36,73720000.0,0.0323
2012-05-24,33.03,32.95,33.21,31.77,50280000.0,0.0322
...,...,...,...,...,...,...
2022-05-11,188.74,196.32,200.67,188.27,31040000.0,-0.0451
2022-05-12,191.24,187.05,198.05,184.78,38110000.0,0.0132
2022-05-13,198.62,192.58,199.89,191.28,24370000.0,0.0386


## 중복데이터 제거하기
* drop_duplicates 를 통해 중복된 데이터가 있다면 제거합니다.

In [135]:
# drop_duplicates : row들 끼리 data를 비교하여 같은 값이 있으면 row중 하나를 삭제
# df.shape를 전/후로 출력하여 삭제된 row가 있는지 확인
df = df.drop_duplicates()
df.shape

(134, 9)

## 기술통계값 구하기

In [136]:
# describe는 기본적으로 수치데이터에 대한 기술통계값을 구하게 됩니다.
df.describe()

,종가,전일비,시가,고가,저가,거래량
count,134.000000,134.000000,134.000000,134.000000,134.000000,1.340000e+02
mean,147305.223881,4729.104478,147328.358209,151842.537313,143483.582090,5.523498e+05
std,30357.353638,4763.691382,29835.873377,32893.174960,28269.900021,1.106149e+06
min,85800.000000,0.000000,85800.000000,86600.000000,85000.000000,8.119600e+04
25%,129375.000000,1500.000000,130625.000000,133000.000000,127500.000000,2.036890e+05
50%,141750.000000,3750.000000,141750.000000,145250.000000,139000.000000,3.094490e+05
75%,169000.000000,6000.000000,169250.000000,173500.000000,166000.000000,5.063350e+05
max,238500.000000,33500.000000,240500.000000,248500.000000,216500.000000,1.179988e+07


* 참고 : E notation
[과학적 기수법 - 위키백과, 우리 모두의 백과사전](https://ko.wikipedia.org/wiki/%EA%B3%BC%ED%95%99%EC%A0%81_%EA%B8%B0%EC%88%98%EB%B2%95)

과학적 기수법, 과학적 표기법(scientific notation, scientific form, standard index form, standard form)은 너무 크거나 너무 작은 숫자들을 십진법으로 편하게 작성하여 표현하는 방법이다. 과학자, 수학자, 공학자들이 공통적으로 사용하는데, 부분적인 이유는 특정한 산술을 단순화시켜 주기 때문이다. 과학 계산기에서는 "SCI" 디스플레이 모드라는 이름으로 알려져 있다.

In [137]:
# 거래량의 과학적 기수법 읽기
5.63350e+05

563350.0

In [138]:
5.063350 * 10 **5

506335.0

## 최근 날짜 구해서 파일명 만들기

In [139]:
# 날짜 column의 첫 row값 확인
# date

date = df.iloc[0]["날짜"]
date

'2022.05.18'

In [140]:
# 종목명, 종목코드, 날짜를 이름으로 하는 csv 파일명 만들기
# file_name

file_name = f"{item_name}_{item_code}_{date}.csv"
file_name

'카카오뱅크_377300_2022.05.18.csv'

## 파일로 저장하기

In [141]:
# 파일로 저장하기 
# index=False 로 데이터프레임의 기본 index 는 저장하지 않도록 합니다.
df.to_csv(file_name, index = False)

In [142]:
# 제대로 저장되었는지 파일을 읽어서 확인합니다.
pd.read_csv(file_name)

,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,377300,카카오뱅크,2022.05.18,90100.0,1400.0,89600.0,90900.0,89300.0,108398.0
1,377300,카카오뱅크,2022.05.17,88700.0,2900.0,85800.0,88700.0,85400.0,214643.0
2,377300,카카오뱅크,2022.05.16,85800.0,200.0,86500.0,88700.0,85700.0,277584.0
3,377300,카카오뱅크,2022.05.13,86000.0,100.0,85900.0,86600.0,85000.0,307400.0
4,377300,카카오뱅크,2022.05.12,85900.0,5500.0,90000.0,90000.0,85000.0,512917.0
...,...,...,...,...,...,...,...,...,...
129,377300,카카오뱅크,2021.11.09,147000.0,6500.0,153500.0,157500.0,146000.0,892617.0
130,377300,카카오뱅크,2021.11.08,153500.0,16500.0,168500.0,169500.0,152000.0,1394625.0
131,377300,카카오뱅크,2021.11.05,170000.0,1000.0,167500.0,179000.0,167500.0,1597937.0
132,377300,카카오뱅크,2021.11.04,169000.0,24000.0,190000.0,191000.0,166000.0,3487030.0


## 전체 과정을 하나의 함수로 만들기

In [143]:
def get_item_list(item_code, item_name):
    """
    일별 시세를 수집하는 함수
    """
    page_no = 1
    # 데이터를 저장할 빈 변수 선언
    item_list = []
    # item_code = "373220"
    # item_name = "LG에너지솔루션"
    item_code = "377300"
    item_name = "카카오페이"
    cnt = 0
    prev = ""

    while True:
        print('*', end = '')
        cnt += 1

        df_one_page = get_day_list(item_code, page_no)
        # 마지막 날짜를 가져옴
        curr = df_one_page.iloc[-1, 0]
        # 마지막 날짜를 비교했을 때 같으면 반복문을 종료
        if curr == prev:
            break

        item_list.append(df_one_page)
        page_no = page_no + 1
        # 현재 날짜를 이전 날짜 변수에 담아서 다음번에 비교합니다.
        prev = curr
    print(cnt)
            
    df = pd.concat(item_list)
    
    df["종목코드"] = item_code
    df["종목명"] = item_name
    
    cols = ['종목코드', '종목명', '날짜', '종가', '전일비', '시가', '고가', '저가', '거래량']
    df = df[cols]
    
    df = df.drop_duplicates()
    
    date = df.iloc[0]["날짜"]
    
    file_name = f"{item_name}_{item_code}_{date}.csv"
    
    df.to_csv(file_name, index = False)
    
    return pd.read_csv(file_name)
    
    
    
    

In [145]:
item_code = "323410"
item_name = "카카오뱅크"
get_item_list(item_code, item_name)

***************15


,종목코드,종목명,날짜,종가,전일비,시가,고가,저가,거래량
0,377300,카카오페이,2022.05.18,89900.0,1200.0,89600.0,90900.0,89300.0,110746.0
1,377300,카카오페이,2022.05.17,88700.0,2900.0,85800.0,88700.0,85400.0,214643.0
2,377300,카카오페이,2022.05.16,85800.0,200.0,86500.0,88700.0,85700.0,277584.0
3,377300,카카오페이,2022.05.13,86000.0,100.0,85900.0,86600.0,85000.0,307400.0
4,377300,카카오페이,2022.05.12,85900.0,5500.0,90000.0,90000.0,85000.0,512917.0
...,...,...,...,...,...,...,...,...,...
129,377300,카카오페이,2021.11.09,147000.0,6500.0,153500.0,157500.0,146000.0,892617.0
130,377300,카카오페이,2021.11.08,153500.0,16500.0,168500.0,169500.0,152000.0,1394625.0
131,377300,카카오페이,2021.11.05,170000.0,1000.0,167500.0,179000.0,167500.0,1597937.0
132,377300,카카오페이,2021.11.04,169000.0,24000.0,190000.0,191000.0,166000.0,3487030.0


## 내가 만든 함수의 기능과 소스코드가 궁금하다면?

In [41]:
# 도움말 보기
# get_item_list?


In [42]:
# 소스코드 보기
# get_item_list??
